In [95]:
import pandas as pd
import os
import json
import numpy as np
import re
from dateutil.parser import parse

In [96]:
## reading the Survey File
#surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\Test.csv",sep="," , skiprows=1)
surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\AmenityClubSurvey FINAL v2 Season Cleaned_deletedFirstTwoColumn.csv",sep="," , skiprows=1, keep_default_na=False)
surveyData.head()

## Reading the Config File and Extracting the Information from there
config_file = open("C:\Projects\MSS\Loading _AmenityData_from_Survey\Python_Codes\ClubCorp\Pool\PoolColMapping.json")
Sauna_config = json.load(config_file)

columnName = ""
months = {1 : "January",2 : "February",3 : "March", 4 : "April", 5 : "May", 6 : "June", 7 : "July", 8 : "August", 9 : "September", 10 : "October", 11 : "November", 12 : "December"}

In [97]:
def determineBooleanValues (dataFrame):
    val = False
    if pd.notnull(dataFrame[columnName]) :
        if dataFrame[columnName] == columnName:
            val = True
    return val

In [98]:
def addToDayTimeMap (phonebook, key, value):
    if key in phonebook:
        x =  phonebook[key]
        phonebook[key] = x + "," + value
        
    else :
        phonebook[key] = value

In [99]:
def prepareHoursOfOperationJson(hoursOfOperationInfo, surveyData, additionalInfoIndex, memorialDayToLaborDayIndex,poolOpenDateIndex,poolCloseDateIndex):
    
    allHoursOfOperation = []
    
    for i in range(0, len(surveyData)) : 
        hoursOfOperationForEachRow = []
        hoursOfOperation = {}
        hoursOfOperationValue = {}
        
        memorialDayToLaborDayInfo = surveyData.iat[i, memorialDayToLaborDayIndex]
        
        season = ""
        seasonStartDate = ""
        seasonStartMonth = ""
        seasonEndDate = ""
        seasonEndMonth = ""
        
        if memorialDayToLaborDayInfo == "Yes - If Yes, then please skip question 46" :
            season = "Memorial Day to Labor Day"
        
        elif memorialDayToLaborDayInfo == "No - If no, then please fill in your pool's open and close dates in question 46":
            
            poolOpenDateInfo = surveyData.iat[i, poolOpenDateIndex]
            poolCloseDateInfo = surveyData.iat[i, poolCloseDateIndex]
            
            poolOpenDateInfoInDateFormat = parse(poolOpenDateInfo)
            seasonStartDate = '{:02d}'.format(poolOpenDateInfoInDateFormat.day)
            seasonStartMonth = months[poolOpenDateInfoInDateFormat.month]
            
            poolCloseDateInfoInDateFormat = parse(poolCloseDateInfo)
            seasonEndDate = '{:02d}'.format(poolCloseDateInfoInDateFormat.day)
            seasonEndMonth = months[poolCloseDateInfoInDateFormat.month]
        
        
        ## adding season value
        if season and not pd.isnull(season):
            seasonValue = {}
            seasonValue["value"] = season
            hoursOfOperationValue["Season"] = [seasonValue]
        
         ## adding season value
        if season and not pd.isnull(season):
            seasonValue = {}
            seasonValue["value"] = season
            hoursOfOperationValue["Season"] = [seasonValue]

        ## Adding season start
        seasonStart = {}
        seasonStartValue = {}
    
        if seasonStartDate and not pd.isnull(seasonStartDate):
            date ={}
            date["value"] = seasonStartDate
            seasonStartValue["Date"] = [date]

        if seasonStartMonth and not pd.isnull(seasonStartMonth):
            month ={}
            month["value"] = seasonStartMonth
            seasonStartValue["Month"] = [month]


        if seasonStartValue :
            seasonStart["value"] = seasonStartValue
            hoursOfOperationValue["SeasonStart"] = [seasonStart]


        ## Adding season end
        seasonEnd = {}
        seasonEndValue = {}

        if seasonEndDate and not pd.isnull(seasonEndDate): 
            date ={}
            date["value"] = seasonEndDate
            seasonEndValue["Date"] = [date]

        if seasonEndMonth and not pd.isnull(seasonEndMonth):
            month ={}
            month["value"] = seasonEndMonth
            seasonEndValue["Month"] = [month]

        if seasonEndValue :
            seasonEnd["value"] = seasonEndValue
            hoursOfOperationValue["SeasonEnd"] = [seasonEnd]
        
        
        ## adding additional information
        additionalInfo = surveyData.iat[i, additionalInfoIndex]    
        if not pd.isnull(additionalInfo) and additionalInfo.lower() != '' and additionalInfo.lower() != '0' and additionalInfo.lower() != 'na' and additionalInfo.lower() != 'n/a' and additionalInfo.lower() != 'no':
            additionalInfoValue = {}
            additionalInfoValue["value"] = additionalInfo
            hoursOfOperationValue["AdditionalInformation"] = [additionalInfoValue]
        
        
        ## creating day time map so that days with same open and closed time can handled properly
        dayTimeMap = {}    
        for j in range(0, len(hoursOfOperationInfo)):
            
            dayInfo = hoursOfOperationInfo[j]
            day = dayInfo["Day"]
            openTimeIndex = dayInfo["OpenTime"]
            closeTimeIndex = dayInfo["CloseTime"]
                
            startTimeRaw = surveyData.iat[i, openTimeIndex]
            endTimeRaw = surveyData.iat[i, closeTimeIndex]
            
            startTime = ""
            endTime = ""
            if pd.notnull(startTimeRaw) and str(startTimeRaw).strip() != '':
                startTime = str(startTimeRaw).strip()
            
            if pd.notnull(endTimeRaw)  and str(endTimeRaw).strip() != '':
                endTime = str(endTimeRaw).strip()
                
            
            
            if (startTime == "Closed" or endTime == "Closed") or (startTime == "" and endTime == "") :
                continue
            else :  
                addToDayTimeMap(dayTimeMap, startTime + "#$" + endTime, day)
            
        
        dayAndTimingArray = []
        
        for startAndEndTime, days in dayTimeMap.items():
            startTime,endTime = startAndEndTime.split("#$")
            openDays = days.split(",")
            ## Adding Day and Timing
            dayAndTiming = {}
            dayAndTimingValue = {}
            for day in openDays :
                dayValue = {}
                dayValue["value"] = True
                dayAndTimingValue[day] = [dayValue]

            timing = {}
            timingValue = {}

            if startTime and not pd.isnull(startTime): 
                startTimeDict ={}
                startTimeDict["value"] = startTime
                timingValue["StartTime"] =  [startTimeDict]

            if endTime and not pd.isnull(endTime):
                endTimeDict ={}
                endTimeDict["value"] = endTime
                timingValue["EndTime"] =  [endTimeDict]
            
            ## Adding day part as "All Day"
            dayPart ={}
            dayPart["value"] = "All Day"
            timingValue["DayPart"] =  [dayPart]


            if timingValue :
                timing["value"] = timingValue
                dayAndTimingValue["Timing"] = [timing]

            if dayAndTimingValue : 
                dayAndTiming["value"] = dayAndTimingValue    
                dayAndTimingArray.append(dayAndTiming)
        
        hoursOfOperationValue["DayAndTiming"] = dayAndTimingArray
        if hoursOfOperationValue :
            hoursOfOperation["value"] = hoursOfOperationValue
            
        hoursOfOperationForEachRow.append(hoursOfOperation)
        allHoursOfOperation.append(json.dumps(hoursOfOperationForEachRow))
    
    return allHoursOfOperation

In [100]:
## Getting the output File name
output_fileName = Sauna_config['outputFileName']


all_column_names = []
Sauna_related_column_names = []

columnIndexForSauna = []

common_column_count = len(Sauna_config['CommonColumn'])

for i in range(0, common_column_count):
    columnIndexForSauna.append(Sauna_config['CommonColumn'][i]['columnIndex'])
    all_column_names.append(Sauna_config['CommonColumn'][i]['outputColumnName'])

Sauna_column_count = len(Sauna_config['PoolRelatedColumn'])

for j in range(0, Sauna_column_count):
    columnIndexForSauna.append(Sauna_config['PoolRelatedColumn'][j]['columnIndex'])
    all_column_names.append(Sauna_config['PoolRelatedColumn'][j]['outputColumnName'])
    Sauna_related_column_names.append(Sauna_config['PoolRelatedColumn'][j]['outputColumnName'])


## Data for Sauna
Sauna = surveyData.iloc[:, columnIndexForSauna]
Sauna.columns = all_column_names

## getting Hours of Related column information from config file
additionalInfoIndex = Sauna_config['AdditionalInfoIndex']
memorialDayToLaborDayIndex = Sauna_config['MemorialDayToLaborDayIndex']
poolOpenDateIndex = Sauna_config['PoolOpenDateIndex']
poolCloseDateIndex = Sauna_config['PoolCloseDateIndex']

hoursOfOperationInfo = Sauna_config['HoursOfOperationInformation']

## Adding Hours of Operation
hrsOfOperation = prepareHoursOfOperationJson(hoursOfOperationInfo, surveyData, additionalInfoIndex, memorialDayToLaborDayIndex,poolOpenDateIndex,poolCloseDateIndex)
Sauna['HrsOfOpeartion'] = hrsOfOperation

Sauna['Index']=1
Sauna['Status'] = 'Active'
Sauna['Type'] = 'Swimming'
Sauna['Name'] = 'Pool'

conditionColumns = []

for x  in Sauna_related_column_names:
    columnName = x
    tempName = columnName + '_Boolean'
    conditionColumns.append(tempName)
    Sauna[tempName] = Sauna.apply(determineBooleanValues, axis=1)

filterCondition = False

for x in conditionColumns :
    tempCon = Sauna[x]
    filterCondition = filterCondition | tempCon

clubsNotHavingFitnessCenter = Sauna[~filterCondition]

clubsNotHavingFitnessCenter.to_csv("C:/Projects/MSS/Loading _AmenityData_from_Survey/Python_Codes/ClubCorp/Pool/Pool_ClubsNotHaving.csv")

Sauna = Sauna[filterCondition]
    
Sauna.to_csv("C:\\Projects\\MSS\\Loading _AmenityData_from_Survey\\Python_Codes\\ClubCorp\\Pool\\" + output_fileName)

C:\Users\akk\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\akk\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\akk\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o